In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
import joblib
from tensorflow.keras.models import load_model
from springstone.data import get_data
import numpy as np

2022-03-10 22:03:07.568896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-10 22:03:07.568953: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
ticker = 'SPY'

In [4]:
preprocessor = joblib.load(f'preprocessor_{ticker}.joblib')

In [5]:
hist = get_data('SPY', start='2022-01-11', end='2022-03-11')
hist.shape

[*********************100%***********************]  1 of 1 completed


(41, 5)

In [6]:
hist_preprocessed = preprocessor.transform(hist)

In [7]:
hist_preprocessed.shape

(40, 26)

In [8]:
hist_preprocessed = hist_preprocessed[-14:,:]

In [9]:
model = load_model(ticker)

2022-03-10 22:04:15.126976: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-10 22:04:15.127021: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-10 22:04:15.127040: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MacBookPro15): /proc/driver/nvidia/version does not exist
2022-03-10 22:04:15.127230: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-10 22:04:16.699625: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 1

In [10]:
hist_preprocessed = np.expand_dims(hist_preprocessed, axis=0)

In [11]:
hist_preprocessed.shape

(1, 14, 26)

In [29]:
scaled_close_prediction = model.predict(hist_preprocessed)
scaled_close_prediction

array([[2.52495]], dtype=float32)

In [30]:
fin_scaller_output = np.zeros((1,7))
fin_scaller_output[0,1] = scaled_close_prediction
fin_scaller_output

array([[0.        , 2.52495003, 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [31]:
unscaled_close_prediction = preprocessor.named_steps['transform'].named_transformers_['fin_scaller'].inverse_transform(fin_scaller_output)[0,1]
unscaled_close_prediction

348.3908930826029

In [33]:
get_data(ticker, start='2022-03-01', end='2022-03-11')[['Close']]

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2022-03-01,429.980011
2022-03-02,437.890015
2022-03-03,435.709991
2022-03-04,432.170013
2022-03-07,419.429993
2022-03-08,416.250000
2022-03-09,427.410004
2022-03-10,425.480011
